Importing Libraries

In [1]:
import time
import re, string
import nltk
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer 
from wordcloud import WordCloud
import os
from google.colab import drive
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import sklearn
from scipy import sparse
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from scipy.sparse import coo_matrix, hstack
lemmatizer = WordNetLemmatizer()
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

Mounting to Google Drive Folder

In [2]:
drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/DBA3803')
os.getcwd()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


'/content/drive/My Drive/DBA3803'

Reading training data

Getting the count frequency of selected features for each comment and appending the counts next to the labels for each comment

Displaying a few rows of the training data

In [3]:
train = pd.read_csv("train.csv")
train[['Comments']] = train[['Comments']].astype(str)
train['white horse'] = train['Comments'].apply(lambda comment: len(re.findall('(?=white horse)', comment.lower())))
train['fuck'] = train['Comments'].apply(lambda comment: len(re.findall('(?=fuck)', comment.lower())))
train['fk'] = train['Comments'].apply(lambda comment: len(re.findall('(?=fk)', comment.lower())))
train['pussy'] = train['Comments'].apply(lambda comment: len(re.findall('(?=pussy)', comment.lower())))
train['boobs'] = train['Comments'].apply(lambda comment: len(re.findall('(?=boobs)', comment.lower())))
train['tmd'] = train['Comments'].apply(lambda comment: len(re.findall('(?=tmd)', comment.lower())))
train['xmm'] = train['Comments'].apply(lambda comment: len(re.findall('(?=xmm)', comment.lower())))
train['fark'] = train['Comments'].apply(lambda comment: len(re.findall('(?=fark)', comment.lower())))
train['sinkies'] = train['Comments'].apply(lambda comment: len(re.findall('(?=sinkies)', comment.lower())))
train['nabei'] = train['Comments'].apply(lambda comment: len(re.findall('(?=nabei)', comment.lower())))
train['ah neh'] = train['Comments'].apply(lambda comment: len(re.findall('(?=ah neh)', comment.lower())))
train['oppies'] = train['Comments'].apply(lambda comment: len(re.findall('(?=oppies)', comment.lower())))
train['capitals'] = train['Comments'].apply(lambda comment: sum(1 for c in comment if c.isupper()))
train['num_exclamation_marks'] = train['Comments'].apply(lambda comment: comment.count('!'))
train['num_smilies'] = train['Comments'].apply(lambda comment: sum(comment.count(w) for w in (':-)', ':)', ';-)', ';)')))
train['pap'] = train['Comments'].apply(lambda comment: comment.lower().count('pap'))
train['pappies'] = train['Comments'].apply(lambda comment: comment.lower().count('pappies'))
train['ang mo'] = train['Comments'].apply(lambda comment: comment.lower().count('ang mo'))
train['dog'] = train['Comments'].apply(lambda comment: len(re.findall('(?=dog)', comment.lower())))
train['knn'] = train['Comments'].apply(lambda comment: len(re.findall('(?=knn)', comment.lower())))
train['cb'] = train['Comments'].apply(lambda comment: len(re.findall('(?=cb)', comment.lower())))
print(len(train))
train.head(5)

20000


,Comments,Insulting,Anti Government,Xenophobic,Racist,Sexual,white horse,fuck,fk,pussy,...,oppies,capitals,num_exclamation_marks,num_smilies,pap,pappies,ang mo,dog,knn,cb
0,FUCK FOREIGN TALENT,1,0,1,0,0,0,1,0,0,...,0,17,0,0,0,0,0,0,0,0
1,I LOVE THE GOVERNMENT,0,0,0,0,0,0,0,0,0,...,0,18,0,0,0,0,0,0,0,0
2,WOMEN SHOULD BE IN THE KITCHEN,1,0,0,0,0,0,0,0,0,...,0,25,0,0,0,0,0,0,0,0
3,I still don't know how he was crushed. What we...,1,1,0,0,0,0,0,0,0,...,0,5,0,0,0,0,0,0,0,0
4,In the first place how did a doctor become a d...,1,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


Function to clean the data and to detect the word's Part of Speech tag in order to accurately lemmatise the word based on its Part of Speech Tag

In [0]:
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

def clean_text(sentences):
    print('Preprocessing Begins...')
    result = []
    for text in sentences:
        text = str(text)
        text = text.lower()
        text = re.sub(r"\'m", " am", text)
        text = re.sub(r"\'s", " is", text)
        text = re.sub(r"\'ll", " will", text)
        text = re.sub(r"\'ve", " have", text)
        text = re.sub(r"\'re", " are", text)
        text = re.sub(r"won\'t", "will not", text)
        text = re.sub(r"can\'t", "cannot", text)
        text = re.sub(r"n\'t", " not", text)
        text = re.sub(r"\'bout", "about", text)
        text = re.sub(r"\'til", "until", text)
        text = re.sub(r"""[^a-zA-Z0-9-!$%^&@*()+|=`{}\[\]:;"'<>?,.\/\s]"""," ",text)
        text = re.sub(r"\bft\b|\bforeigntalent\b","foreign talent",text)
        text = re.sub(r"\bgovt\b|\bgahment\b|\bgahmen\b|\bgov\b","government",text)
        text = re.sub(r"\bpappies\b|\bpapies\b|\bpapees\b|\bpap government\b","pap",text)
        text = re.sub(r"\bchi bai\b|\bchibai\b|\bchee bye\b|\bcheebye\b","cb",text)
        text = re.sub(r"\bfk\b|\bfck\b","fuck",text)
        text = re.sub(r"\bkanina\b|\bkanena\b","knn",text)
        text = re.sub(r"\blky\b","lee kuan yew",text)
        text = re.sub(r"\blhl\b|\bpm lee\b","lee hsien loong",text)
        if len(text.split()) == 0:
            continue
        then = []
        sw = stopwords.words('english')
        for word in text.split():
            if word not in sw:
              word = lemmatizer.lemmatize(word, get_wordnet_pos(word))
              then.append(word)
        result.append(then)
    result = [" ".join(x) for x in result]
    print(result[:3], '\n')
    return result



Cleaning the data with the earlier function

In [5]:
clean_train = clean_text(train["Comments"])
clean_join = pd.Series(clean_train)
train["Comments"] = clean_join
train = train.dropna()
train["Comments"][:5]

Preprocessing Begins...
['fuck foreign talent', 'love government', 'woman kitchen'] 



0                                  fuck foreign talent
1                                      love government
2                                        woman kitchen
3    still know crushed. 3 material time accident h...
4    first place doctor become defense minister mil...
Name: Comments, dtype: object

Creating a list for all the labels

In [0]:
label_cols = ['Insulting', 'Anti Government', 'Xenophobic', 'Racist', 'Sexual']

Creating an array of all the newly added features and their count frequencies and a list of all the new added feature names

In [0]:
features = np.array(train.iloc[:,6:])
feature_names = train.columns.values[6:]
feature_names = list(feature_names)

Using the TfidfVectorizer function to create a Term Frequency-Inverse Document Frequency Document Term Matrix.

Creating Bigrams and Unigrams.

Feature must at least appear in 3 comments to be in the Matrix.

Appending the frequency counts as features next to the TF-IDF values of each feature

Displaying some of the features in the Matrix.

In [8]:
print(len(train["Comments"]))
n = train.shape[0]
vec = TfidfVectorizer(ngram_range=(1,2), 
               min_df=3, max_df=0.9, strip_accents='unicode')
trn_term_doc = vec.fit_transform(train["Comments"])
trn_term_doc = sparse.csr_matrix(hstack([trn_term_doc,features]).toarray())
print(vec.get_feature_names()[5000:5050])
print(len(vec.get_feature_names()))

20000
['laobu', 'laocheebye', 'laoyeh', 'lapse', 'laptop', 'lar', 'lara', 'large', 'lash', 'last', 'last day', 'last friday', 'last minute', 'last month', 'last night', 'last one', 'last pic', 'last post', 'last stick', 'last time', 'last week', 'last year', 'last years', 'lat', 'late', 'late night', 'late smrt', 'late work', 'lately', 'later', 'later ah', 'later go', 'later ish', 'later one', 'later well', 'latha', 'lau', 'laugh', 'laugh stock', 'launch', 'laundering', 'laundry', 'laundry rack', 'laus', 'lavender', 'law', 'law abiding', 'laws', 'lawson', 'lawyer']
10453


Creating a likelihood function whereby it sums the TF-IDF weights of a feature given the comment contains the class and divides it by the total number of comments that contain that respective class.

Displaying the new TF-IDF Document - Term matrix with newly added count frequencies of some features from the Feature Engineering

In [15]:
def pr(y_i, y):
    p = x[y==y_i].sum(0)
    return (p+1) / ((y==y_i).sum()+1)
  
x = trn_term_doc

columns = [vec.get_feature_names()]
columns = [x for y in columns for x in y]
columns = [*columns, *feature_names]
pd.SparseDataFrame(x[:5,1000:], columns = columns[1000:])

,belongs,beloved,below,bend,benefit,beng,beng ah,bengs,beo,beret,...,oppies,capitals,num_exclamation_marks,num_smilies,pap,pappies,ang mo,dog,knn,cb
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,17.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,18.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,25.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Creating a function where it log divides both likelihoods to create the Bayesian ratio and multiplies the ratio against the TF-IDF Document Term matrix

Function creates a Logistic Regression model for each of the 5 labels with each label having their own y variable (either 1 or 0) and x variable (ratio adjusted TF-IDF Document Term Matrix)

In [0]:
scores = []
sd = []

def get_mdl(y):
    y = y.values
    r = np.log(pr(1,y) / pr(0,y))
    m = LogisticRegression(solver = "liblinear", max_iter = 10000000)
    x_nb = x.multiply(r)
    cv_score = np.mean(cross_val_score(
        m, x_nb, y, cv=5, scoring='roc_auc'))
    cv_sd = np.std(cross_val_score(
        m, x_nb, y, cv=5, scoring='roc_auc'))
    scores.append(cv_score)
    sd.append(cv_sd)
    print('CV score for class {} is {}, standard deviation is {}'.format(j, cv_score, cv_sd))
    return m.fit(x_nb, y)

Running earlier functions to create Bayesian ratio and to fit the ratio multiplied TF-IDF Document Term matrix and the classes (either 1 or 0) to 5 different logistic regression models representing each of the 5 labels.

Using 5-fold Cross Validation scoring by the Receiver Operating Characteristic, Area Under Curve measure we can gather the mean of the score and the mean of the standard deviation for each label. With these 5 scores and standard deviations we mean them once again to get the CV score and standard deviation that is representative of all labels.

In [11]:
for i, j in enumerate(label_cols):
    print('fit', j)
    m = get_mdl(train[j])

print('Total CV score is {}'.format(np.mean(scores)))

print('The Standard Deviation of the CV score is {}'.format(np.std(scores)))


fit Insulting
CV score for class Insulting is 0.7985194355116075, standard deviation is 0.033424584397716464
fit Anti Government
CV score for class Anti Government is 0.8918991934337731, standard deviation is 0.04989302851647485
fit Xenophobic
CV score for class Xenophobic is 0.9011563786433511, standard deviation is 0.02477832936339792
fit Racist
CV score for class Racist is 0.8773421618083728, standard deviation is 0.04192834417023056
fit Sexual
CV score for class Sexual is 0.8459029023063165, standard deviation is 0.04686937654491669
Total CV score is 0.8629640143406843
The Standard Deviation of the CV score is 0.03727416716463475


Creating similar function as the one earlier, except without CV scores and standard deviations

In [0]:
def test_mdl(y):
    y = y.values
    r = np.log(pr(1,y) / pr(0,y))
    m = LogisticRegression(C=4, dual=True, solver = "liblinear", max_iter = 100000)
    x_nb = x.multiply(r)
    return m.fit(x_nb, y), r

Running a test comment on the model and getting the run time for the test comment's results to be generated

In [13]:
start = time.time()

test_str = "You are stupid"

test_com = pd.DataFrame([test_str])
test_com.columns = ["Comments"]


test_com['white horse'] = test_com['Comments'].apply(lambda comment: len(re.findall('(?=white horse)', comment.lower())))
test_com['fuck'] = test_com['Comments'].apply(lambda comment: len(re.findall('(?=fuck)', comment.lower())))
test_com['fk'] = test_com['Comments'].apply(lambda comment: len(re.findall('(?=fk)', comment.lower())))
test_com['pussy'] = test_com['Comments'].apply(lambda comment: len(re.findall('(?=pussy)', comment.lower())))
test_com['boobs'] = test_com['Comments'].apply(lambda comment: len(re.findall('(?=boobs)', comment.lower())))
test_com['tmd'] = test_com['Comments'].apply(lambda comment: len(re.findall('(?=tmd)', comment.lower())))
test_com['xmm'] = test_com['Comments'].apply(lambda comment: len(re.findall('(?=xmm)', comment.lower())))
test_com['fark'] = test_com['Comments'].apply(lambda comment: len(re.findall('(?=fark)', comment.lower())))
test_com['sinkies'] = test_com['Comments'].apply(lambda comment: len(re.findall('(?=sinkies)', comment.lower())))
test_com['nabei'] = test_com['Comments'].apply(lambda comment: len(re.findall('(?=nabei)', comment.lower())))
test_com['ah neh'] = test_com['Comments'].apply(lambda comment: len(re.findall('(?=ah neh)', comment.lower())))
test_com['oppies'] = test_com['Comments'].apply(lambda comment: len(re.findall('(?=oppies)', comment.lower())))
test_com['capitals'] = test_com['Comments'].apply(lambda comment: sum(1 for c in comment if c.isupper()))
test_com['num_exclamation_marks'] = test_com['Comments'].apply(lambda comment: comment.count('!'))
test_com['num_smilies'] = test_com['Comments'].apply(lambda comment: sum(comment.count(w) for w in (':-)', ':)', ';-)', ';)')))
test_com['pap'] = test_com['Comments'].apply(lambda comment: comment.lower().count('pap'))
test_com['pappies'] = test_com['Comments'].apply(lambda comment: comment.lower().count('pappies'))
test_com['ang mo'] = test_com['Comments'].apply(lambda comment: comment.lower().count('ang mo'))
test_com['dog'] = test_com['Comments'].apply(lambda comment: len(re.findall('(?=dog)', comment.lower())))
test_com['knn'] = test_com['Comments'].apply(lambda comment: len(re.findall('(?=knn)', comment.lower())))
test_com['cb'] = test_com['Comments'].apply(lambda comment: len(re.findall('(?=cb)', comment.lower())))

test_str =  clean_text(test_com["Comments"])
test_join = pd.Series(test_str)
train["Comments"] = test_join

test_features = np.array(test_com.iloc[:,1:])
test_vec = vec.transform(test_com['Comments'])
test_vec = sparse.csr_matrix(hstack([test_vec,test_features]).toarray())

test_pred = np.zeros((len(test_com), len(label_cols)))

for i, j in enumerate(label_cols):
    m,r = test_mdl(train[j])
    test_pred[:,i] = m.predict_proba(test_vec.multiply(r))[:,1]
    if test_pred[:,i] >= 0.5:
      print("\n","This comment is",j)
    
    
score = pd.DataFrame(test_pred, columns = label_cols)
print("\n\n",score)


test_com

end = time.time()
print("\n\n","The run time is:",end-start)

Preprocessing Begins...
['stupid'] 


 This comment is Insulting


    Insulting  Anti Government  Xenophobic    Racist    Sexual
0   0.649283         0.001478    0.004745  0.000987  0.017018


 The run time is: 318.2194423675537
